<a href="https://colab.research.google.com/github/marcoandre1/introduction-colab/blob/main/TP1_PartieA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 : Le jeu de données

## 1.1 Télécharger le jeu de données

Le jeu de données est sous la forme d'un fichier CSV.

**Exécutez la cellule ci-dessous pour télécharger le fichier et l'ajouter à votre répertoire actif _Google Colab_.**

In [ ]:
import urllib.request

url = 'https://drive.google.com/uc?export=download&id=13ZfF8DjSvqPJkrY93GJTK2l14IwLVLZ0'
urllib.request.urlretrieve(url, 'IT_Support_Tickets_FR_200.csv')

print('Téléchargement terminé !')

## 1.2 Créer un _DataFrame_ pour inspecter le jeu de données

Un _DataFrame_ est un objet pour représenter un jeu de données venant du module _pandas_. Ce module _Python_ est populaire en IA.

Un _DataFrame_ est une interface conviviale pour traiter un jeu de donnée.

**Suivez les consignes ci-dessous**
1. importez le module `pandas` et donné lui l'alias `pd`
2. créez le _DataFrame_
  * créez une nouvelle variable nommée `df`
  * utilisez la fonction suivante `pd.read_csv("IT_Support_Tickets_FR_200.csv")` pour créer le _DataFrame_ en lisant le jeu de données téléchargé. Affectez la valeur retournée à la variable `df`.
3. finalement, ajoutez une dernière ligne qui n'est que le suivant : `df`. Lorsqu'une cellule termine qu'avec le nom d'une variable _DataFrame_, _Colab_ va afficher la _DataFrame_.

In [ ]:
## Écrivez votre code dans cette cellule ##


## 1.3 Les types de problèmes
Pour commencer, nous allons nous concentrer sur la **classification par type de problème**.

**Exécutez la celulle ci-dessous pour créer un nouveau _DataFrame_ contenant que le type de problème pour chaque exemple du jeu de donnée**.

In [ ]:
colonne_type_problème = df['Type de problème']

colonne_type_problème

Analysons rapidement le jeu de donnée ; apprendre à le connaître peut nous aider à interpréter des résultats avenirs.

Nous voulons connaître le nombre d'exemple par classe, par exemple.

**Exécutez la cellule ci-dessous pour créer et afficher un nouveau _DataFrame_ avec le compte de chaque type de problème.**

Notez qu'il y a un _déséquilibre des classes_ (ex: il y a beaucoup plus de problèmes 'logiciel' que les autres). Ceci n'est pas idéal, mais nous allons continuer tout de même.

In [ ]:
compte_problème = colonne_type_problème.value_counts()

compte_problème

# 2 : Prétraitement du jeu de données

## 2.1 Extraire les exemples et colonnes pertinents

Souvent, nous n'avons pas besoin de toute l'information d'un jeu. Pour notre problème, nous n'avons pas besoin des colonnes «ID Ticket», «Date», «Client», et «Priorité».

En ce qui concerne les exemples, _tous les exemples_ nous intéressent.

**Complétez la cellule ci-dessous en créant la liste `colonne_à_retirer`. La liste doit contenir le nom des colonnes à retirer.**

La fonction `.drop(.)` qui s'applique sur notre _DataFrame `df` va retourner un nouveau _DataFrame_ sans les colonnes fournies.

In [ ]:
colonne_à_retirer = # Complétez ici
df_classification = df.drop(columns=colonne_à_retirer)

df_classification

## 2.2 Encodage des classes
Allons fournir une séquence à notre modèle et recevoir une classification comme réponse.

Nous avions vu comment la séquence de texte se fera transformer en une sequence vecteur de nombre pour le modèle (tokenizer et embedding).

Il faut aussi que la classe soit un nombre. Présentement, nos classes sont du texte.

Nous allons **encoder** chaque classe en entier.

Le module `sklearn.preprocessing` contient un sous-module `LabelEncoder` qui peut facilement faire cette tâche.

**Exécutez la cellule ci-dessous**. Un objet `encodeur` est créé et utilisé pour créer un nouveau _DataFrame_ `nouvelle_colonne` contenant les encodages. Cette nouvelle colonne est ajouté à notre `df_classification` avec le nom `Classe`. Il est simple d'ajouter une colonne à un _DateFrame_ !

In [ ]:
from sklearn.preprocessing import LabelEncoder

encodeur = LabelEncoder()

nouvelle_colonne = encodeur.fit_transform(df_classification['Type de problème'])
df_classification['Classe'] = nouvelle_colonne

df_classification

## 2.3 Splits _train_, _validation_, _test_
Dans le cadre de l'entraînement d'un modèle de classification, il est essentiel de diviser les données en trois ensembles distincts : train, validation et test.

Le jeu d'entraînement (train) permet au modèle d'apprendre les relations entre les données d'entrée et les étiquettes.

Le jeu de validation est utilisé pour ajuster les hyperparamètres et évaluer les performances du modèle pendant l'entraînement, afin d'éviter le surapprentissage (overfitting).

Enfin, le jeu de test permet de mesurer objectivement la performance finale du modèle sur des données totalement inédites. Cette séparation garantit une évaluation fiable et réaliste de la capacité du modèle à généraliser à de nouveaux cas, ce qui est fondamental pour toute application en production.

Nous allons faire des splits 60/20/20 : 60% du jeu de données sera consacré au jeu d'entraînement, 20% au jeu de validation, et le dernier 20% au jeu de tests.

**Exécuter la cellule ci-dessous** pour créer les splits à l'aide de la fonction `train_test_split` du module `sklearn.model_selection`.

Vous obtiendrez 6 nouveau _DataFrame_ :

*   120 descriptions pour l'entraînement, 40 descriptions pour la validation, et 40 descriptions pour le test
*   120 classes pour l'entraînement, 40 classes pour la validation, et 40 classes pour le test

Ils se nomment `X_train`, `X_val`, `X_test` et `y_train`, `y_val`, `y_test`.

In [ ]:
from sklearn.model_selection import train_test_split

# caractéristiques et étiquettes
X = df_classification['Description'] # nouveau DataFrame avec les descriptions
y = df_classification['Classe'] # nouveau DataFrame avec les classes

# premier split : 20% test, 80% temporaire
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# deuxième split: train + validation (à partir de temporaire)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

print(len(X_train), len(X_val), len(X_test))
print(len(y_train), len(y_val), len(y_test))

# 3 : _Tokenize_ les descriptions des 3 splits

## 3.1 De _DataFrame_ à listes _Python_
Nous allons bientôt utiliser un _tokenizer_ pour transformer les descriptions en séquence de _token_. Le _tokenizer_ fonctionnera sur une liste de texte, et non sur un _DataFrame_.

**Complétez la cellule ci-dessous pour créer trois listes (entraînement, validation et test). Notez comment la première est réalisée pour vous.**

In [ ]:
X_train_list = X_train.to_list()
X_val_list = # complétez ici
X_test_list = # complétez ici

## 3.2 Créer le _tokenizer_ pour _BARThez_

**Dans la cellule ci-dessous, importez le module nécessaire et créer le _tokenizer_.**

Ce même objet pourra créer les _tokens_ pour les 3 splits.

In [ ]:
# importer le module

# créer le tokenizer
#tokenizer = # à compléter

## 3.3 _Tokenize_ les listes

**Utilisez `tokenizer(.)` pour créer les _tokens_ d'entraînement, de validation, et de test.**

Donnez premièrement en paramètre la liste à _tokenize_, ensuite donnez les paramètres suivants : `padding=True, truncation=True, return_tensors="pt"`.

Exemple : `tokenizer(ma_liste, padding=True, truncation=True, return_tensors="pt")`

_L'importance de `padding` et `truncation` ne seront pas expliqué pour le moment._

In [ ]:
train_tokens = # complétez ici
val_tokens = # complétez ici
test_tokens = # complétez ici

**Dans la cellule ci-dessous, `print(.)` la séquence de tokens d'entraînement pour valider que le _tokenizer_ a bien fonctionné.**

Vous devriez voir un _dictionnaire_ contenant un _tensor_ `input_ids` et un autre _tensor_ `attention_mask`.

Un dictionnaire est un type natif à _Python_ qui permet d'associer des valeurs à des mots clef. C'est une structure de données qui permet une organisation simple de plusieurs valeurs.

In [ ]:
# complétez ici

# 4 : Créer le modèle de classification

## 4.1 Importer le module

**Importez le sous-module `AutoModelForSequenceClassification` du module `transformers`**.

Ce module nous permettra d'instancier un modèle de classification à partir de séquence de texte. Le modèle _BARThez_ est un modèle séquence à séquence, mais l'`AutoModelForSequenceClassification` permettra de créer un modèle de classification en utilisant l'encodeur préentraîné de _BARThez_.



In [ ]:
# complétez ici

## 4.2 Instanciation du modèle

Instanciez le modèle à l'aide de la fonction `AutoModelForSequenceClassification.from_pretrained(.)`.

En plus de donner le nom du modèle préentraîné à charger, il faut indiquer le nombre de classes possibles via le paramètre `num_labels = n_classes`.

**Obtenez premièrement le nombre de classes et ensuite instanciez le modèle.
Ne _hardcode_ pas le nombre de classes (5). Utilisez `len(.)` pour obtenir le nombre de classe de façon dynamique.**

In [ ]:
n_classes = # complétez ici
model = AutoModelForSequenceClassification.from_pretrained("moussaKam/barthez", num_labels=n_classes)

# 5 : Création de _DataSet HuggingFace_

## 5.1 Classe personnalisée pour nos données
Nous utilisons l'API d'_HuggingFace_ pour la creation du modèle de classification. Nous allons aussi l'utiliser pour l'entraînement du classifieur.

l'API s'attend que nos données soient représentées dans des objets _PyTorch DataSet_.

Il faut donc prendre nos dictionnaires `train_tokens`, `val_tokens` et `test_tokens` et nos _DataFrame_ `y_train`, `y_val` et `y_test` pour former 3 _DataSet_.

**Exécutez la cellule ci-dessous pour déclarer une _classe Python_ qui représentera un _PyTorch DataSet_ de tickets de TI.**

In [ ]:
from torch.utils.data import Dataset
import torch

class TicketDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels  # This should be a flat list of ints

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)  # Force scalar tensor
        return item


## 5.2 Créer les _DataSet_
**Complétez la cellule ci-dessous afin d'instancier les trois _DataSet_.**

In [ ]:
train_dataset = TicketDataset(train_tokens, y_train.tolist())
val_dataset = # complétez ici
test_dataset = # complétez ici

# 6 : Entraînement du classifieur

## 6.1 Configurer l'entraînement
**Exécutez la cellule suivante**. Celle-ci créer un objet de configuration `training_args` qui contient des hyperparamètres pour l'entraînement (ex: _learning_rate_, _weight_decay_, etc.) et des informations pour le _logging_ avec _Weights & Biases_.

Cette configuration sauvegarde les paramètres optimisés du modèle après chaque itération/époque.

Dans ce cas, une itération contient 15 mises-à-jour/_steps_.

Nous optimiserons le modèle pour 5 itérations, soit un total de 75 _steps_.

Vous obtiendrez un répertoire contenant chaque sauvegarde. Une sauvegarde se nomme un _checkpoint_, en IA.

<img src="https://i.ibb.co/4gNnBHb6/checkpoints.png" width="20%">

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="wandb",
    metric_for_best_model="f1",
    logging_dir="./logs",
)

model.config.early_stopping = None
model.config.num_beams = None
model.config.no_repeat_ngram_size = None

## 6.2 Fonction pour évaluer la précision et score F1
**Exécuter la cellule ci-dessous pour déclarer la fonction qui sera utilisée par l'entraîneur afin d'évaluer la précision et le score F1 du modèle.**

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    if isinstance(predictions, tuple):
        predictions = predictions[0]

    preds = np.argmax(predictions, axis=1)

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

## 6.3 Créer l'entraîneur avec l'API `Trainer` d'_HuggingFace_

**Exécutez la cellule ci-dessous pour créer un objet `trainer`**. Celui-ci indique quel modèle entraîné la configuration ci-dessus et les jeux `train_dataset` et `val_dataset`.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

## 6.4 Entraîner le modèle !
**Exécutez la cellule ci-dessous pour démarrer l'entraînement.**

Le paramètre `resume_from_checkpoint=False` est pour indiquer à l'entraîneur «recommencer de 0» l'entraînement, au lieu de continuer l'entraînement à partir du dernier _checkpoint_.

**Après l'entraînement**, allez sur le projet «INF717-TP1» dans votre compte _Weights and Biases_ et observez les résultats. Les graphiques les plus importants pour nous sont :

*   eval/loss
*   train/loss
*   eval/f1
*   eval/accuracy

In [ ]:
import wandb
wandb.init(project="INF717-TP1")
trainer.train(resume_from_checkpoint=False)

# 7 : Test du modèle

**Exécutez les cellules suivantes pour évaluer le modèle sur le jeu de test.**

In [ ]:
test_metrics = trainer.evaluate(test_dataset)
print(test_metrics)

In [ ]:
predictions = trainer.predict(test_dataset)

logits = predictions.predictions
if isinstance(logits, tuple):
    logits = logits[0]

pred_labels = np.argmax(logits, axis=1)
true_labels = predictions.label_ids

for i in range(len(test_dataset)):
    print("Description:", X_test.iloc[i])
    print("True label:", encodeur.inverse_transform([true_labels[i]])[0])
    print("Predicted:", encodeur.inverse_transform([pred_labels[i]])[0])
    print("-" * 50)